In [1]:
import pandas as pd
import torch
from gensim.models.word2vec import Word2Vec
import re
from collections import Counter

In [15]:
class Embedded_Model():
    def __init__(self,path,add_path):
        self.DF=pd.read_csv(path,index_col=0)
        self.data=self.DF['name']
        self.DF1=pd.read_csv(add_path,index_col=0)
        self.data1=self.DF1['name']
        self.data=pd.concat([self.data,self.data1])
        self.data=self.process()
    def process(self):
        self.data=self.data.apply(lambda x:re.sub(r'.*\d+회','',x).strip())
        for punc in '[!\"#$%&\'()*+,-./:;<=>?@\[\]^_`{|}~]':
            self.data=self.data.apply(lambda x: str.replace(x,punc,' '))
        return self.data.apply(lambda x: [y for y in x.split(' ') if len(y)>=2])
    def word2vec(self):
        wordvec=Word2Vec(self.data.tolist(),vector_size=256,alpha=0.02,negative=10,window=3,min_count=1,sg=1,epochs=200,max_final_vocab=None)
        wordvec.save('word2vec.model')
        # torch.save(torch.tensor([wordvec.wv[word] for word in wordvec.wv.index_to_key]),'word2vec.model')
        # print(1)
        return {y:x for x,y in enumerate(wordvec.wv.index_to_key)}
    
class Predict_name():
    def __init__(self,path,model):
        self.DF=pd.read_csv(path,index_col=0)
        self.word2vec=Word2Vec.load(model)
    def pred(self,pred_name,number:int=None):
        if number==None:
            number=5
        try:
            self.pred_name_list=[x for x in self.word2vec.wv.index_to_key if pred_name.lower() in x.lower()]
        except:
            self.pred_name_list=[]
        print(self.pred_name_list)
        if len(self.pred_name_list)!=0:
        # self.pred_name=pred_name.lower()
            self.name=self.DF['name']
            total_match_list=[]
            for pred_name in self.pred_name_list:
                self.word_list=[x for x,y in self.word2vec.wv.most_similar(pred_name,topn=number)]
                match_list=[]
                for word in self.word_list:
                    match=self.DF.loc[self.name.apply(lambda x: True if word in x.lower() else False),'name'].tolist()
                    match_list.extend(match)
                total_match_list.extend(match_list)
            dict1=dict(Counter(total_match_list))
            return dict(sorted(dict1.items(),key=lambda x:x[1],reverse=True))
        else:
            return '키워드 없음'
            


In [3]:
emb=Embedded_Model('./info.csv','./add_info.csv')

In [4]:
word2vec=emb.word2vec()

In [158]:
w=Word2Vec.load('./word2vec.model')
[x for x in w.wv.index_to_key if '공군' in x]

['공군부대']

In [16]:
model=Predict_name('./info.csv','./word2vec.model')
pred_=model.pred('공군',number=5)

['공군부대']


In [17]:
pred_

{'공군부대 공간혁신구역 선도사업지 시민 아이디어 공모전': 2}